In [1]:
!pip install mediapy
!pip install -U gdown

In [1]:
import os
from zipfile import ZipFile 
import mediapy as media

# pytorch dataset
from torch.utils.data import Dataset
import torch
import torchvision.transforms as transforms
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict

## Get raw videos from google drive

In [3]:
!gdown https://drive.google.com/drive/folders/1Y8W5yiAxTDiRzx_9-caqa9dNhmo4dYcS --folder 

Retrieving folder contents
Retrieving folder 11BvoNE9dgq-qPrLqQED5P1N3UU9EP8vN data
Retrieving folder 1Sv_zHg8Ds2B7RDphAZVRpO2R-fpBnwz7 raw videos
Processing file 1ENHn_C8em0Y0pRAEMXU5kvIQ-SZm8Qe8 Dataset_AI_Masterchallange 2025-20250424T063117Z-001.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1ENHn_C8em0Y0pRAEMXU5kvIQ-SZm8Qe8
From (redirected): https://drive.google.com/uc?id=1ENHn_C8em0Y0pRAEMXU5kvIQ-SZm8Qe8&confirm=t&uuid=dab1f0ba-c7af-4fc0-a00b-d3c0ad47658b
To: /home/giedriusmirklys/medication-intake-detection-master-challenge/AI in PWF/data/raw videos/Dataset_AI_Masterchallange 2025-20250424T063117Z-001.zip
100%|██████████████████████████████████████| 1.99G/1.99G [01:30<00:00, 22.1MB/s]
Download completed


In [2]:
PATH_CWD = os.getcwd()
PATH_DATA = os.path.join(PATH_CWD + "/AI in PWF/data/")

In [3]:
PATH_DATA

'/home/giedriusmirklys/medication-intake-detection-master-challenge/AI in PWF/data/'

In [5]:
video_zip_file = PATH_DATA + "raw videos/Dataset_AI_Masterchallange 2025-20250424T063117Z-001.zip"
with ZipFile(video_zip_file, "r") as video_zip:
    video_zip.extractall(path=PATH_DATA+"raw videos")

In [6]:
# os.remove(video_zip_file)

## Helper functions

In [4]:
def play_video(fname):
    fname = PATH_DATA + fname
    video = media.read_video(fname)
    media.show_video(video, title=fname.split("/")[-1].split(".")[0], fps=60, width=500)

In [ ]:
play_video("raw videos/Dataset_AI_Masterchallange 2025/video_20250328_011625.mp4")

# Dataset

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, data_dir, frames_per_video: int = 60, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.video_files, self.video_labels = self._load_video_files(data_dir)
        
    def _load_video_files(self, data_dir):
        self.video_dir = os.path.join(data_dir, "raw videos")

        video_files = [name.split("/")[-1] for name in os.listdir(self.video_dir) if name.endswith(".mp4")]

        labels_df = pd.read_csv(os.path.join(data_dir, "annotations_mid.csv"))

        labels = defaultdict(tuple)
        for _, row in labels_df.iterrows():
            labels[row["filename"]] = (row["label"], row["edge_case"])
        
        return video_files, labels
    
    def __getitem__(self, index):
        assert self.set_mean is not None, "Please call get_statistics() before __getitem__(), or set them manually"
        assert self.set_std is not None, "Please call get_statistics() before __getitem__(), or set them manually"

        frames, label, edge_case = self.__load(index)
        # Normalize frames
        frames = (frames - self.set_mean) / self.set_std
        if self.transform:
            frames = [self.transform(frame) for frame in frames]

        return frames, label, edge_case
    
    def __len__(self):
        return len(self.video_files)
    
    def __load(self, index):
        video_path = os.path.join(self.video_dir, self.video_files[index])
        label, edge_case = self.video_labels[self.video_files[index]]
        cap = cv2.VideoCapture(video_path)

        ret = True
        frames = []
        while ret:
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
        # If the video has less than 60 frames, pad with zeros
        if len(frames) < 60:
            frames += [np.zeros_like(frames[0])] * (60 - len(frames))
        # If the video has more than 60 frames, sparse it
        # to 60 frames by taking every nth frame
        elif len(frames) > 60:
            step = len(frames) // 60
            frames = [frames[i] for i in range(0, len(frames), step)]
        
        # Convert to numpy array
        frames = np.array(frames)
        cap.release()
        return frames, label, edge_case
    
    def get_statistics(self):
        data_size = len(self.video_files)
        total_sum = np.zeros(3)
        total_sq_sum = np.zeros(3)
        total_count = np.zeros(3)

        for i in range(data_size):
            frames, _, _ = self.__load(i)
            frames_np = np.array(frames)  # shape: (num_frames, H, W, C)
            # Reshape to (-1, 3) to flatten all pixels, then sum per channel
            pixels = frames_np.reshape(-1, frames_np.shape[-1])
            total_sum += pixels.sum(axis=0)
            total_sq_sum += (pixels ** 2).sum(axis=0)
            total_count += pixels.shape[0]

        self.set_mean = total_sum / total_count
        self.set_std = np.sqrt(total_sq_sum / total_count - self.set_mean ** 2)
            


In [24]:
_set = VideoDataset(data_dir=PATH_DATA)

In [ ]:
_set.get_statistics()